In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the mainDS.csv.
import pandas as pd
application_df = pd.read_csv("/Users/batoogz/Desktop/Module Challenges/project-4-web-scrapers/Resources/mainDS.csv")
application_df.head()

,ID,Loan Amount,Funded Amount,Funded Amount Investor,Term,Batch Enrolled,Interest Rate,Grade,Sub Grade,Home Ownership,...,Recoveries,Collection Recovery Fee,Collection 12 months Medical,Application Type,Last week Pay,Accounts Delinquent,Total Collection Amount,Total Current Balance,Total Revolving Credit Limit,Loan Status
0,65087372,10000,32236,12329.36286,59,BAT2522922,11.135007,B,C4,MORTGAGE,...,2.498291,0.793724,0,INDIVIDUAL,49,0,31,311301,6619,0
1,1450153,3609,11940,12191.99692,59,BAT1586599,12.237563,C,D3,RENT,...,2.377215,0.974821,0,INDIVIDUAL,109,0,53,182610,20885,0
2,1969101,28276,9311,21603.22455,59,BAT2136391,12.545884,F,D4,MORTGAGE,...,4.316277,1.020075,0,INDIVIDUAL,66,0,34,89801,26155,0
3,6651430,11170,6954,17877.15585,59,BAT2428731,16.731201,C,C3,MORTGAGE,...,0.107020,0.749971,0,INDIVIDUAL,39,0,40,9189,60214,0
4,14354669,16890,13226,13539.92667,59,BAT5341619,15.008300,C,D4,MORTGAGE,...,1294.818751,0.368953,0,INDIVIDUAL,18,0,430,126029,22579,0


In [2]:
application_df.keys()

Index(['ID', 'Loan Amount', 'Funded Amount', 'Funded Amount Investor', 'Term',
       'Batch Enrolled', 'Interest Rate', 'Grade', 'Sub Grade',
       'Home Ownership', 'Employment Duration (years)', 'Verification Status',
       'Payment Plan', 'Loan Title', 'Debit to Income',
       'Delinquency - two years', 'Inquires - six months', 'Open Account',
       'Public Record', 'Revolving Balance', 'Revolving Utilities',
       'Total Accounts', 'Initial List Status', 'Total Received Interest',
       'Total Received Late Fee', 'Recoveries', 'Collection Recovery Fee',
       'Collection 12 months Medical', 'Application Type', 'Last week Pay',
       'Accounts Delinquent', 'Total Collection Amount',
       'Total Current Balance', 'Total Revolving Credit Limit', 'Loan Status'],
      dtype='object')

In [3]:
# Drop the non-beneficial ID columns, 'ID'.
application_df = application_df.drop({'ID'},axis=1)
application_df.head()

,Loan Amount,Funded Amount,Funded Amount Investor,Term,Batch Enrolled,Interest Rate,Grade,Sub Grade,Home Ownership,Employment Duration (years),...,Recoveries,Collection Recovery Fee,Collection 12 months Medical,Application Type,Last week Pay,Accounts Delinquent,Total Collection Amount,Total Current Balance,Total Revolving Credit Limit,Loan Status
0,10000,32236,12329.36286,59,BAT2522922,11.135007,B,C4,MORTGAGE,20,...,2.498291,0.793724,0,INDIVIDUAL,49,0,31,311301,6619,0
1,3609,11940,12191.99692,59,BAT1586599,12.237563,C,D3,RENT,4,...,2.377215,0.974821,0,INDIVIDUAL,109,0,53,182610,20885,0
2,28276,9311,21603.22455,59,BAT2136391,12.545884,F,D4,MORTGAGE,10,...,4.316277,1.020075,0,INDIVIDUAL,66,0,34,89801,26155,0
3,11170,6954,17877.15585,59,BAT2428731,16.731201,C,C3,MORTGAGE,12,...,0.107020,0.749971,0,INDIVIDUAL,39,0,40,9189,60214,0
4,16890,13226,13539.92667,59,BAT5341619,15.008300,C,D4,MORTGAGE,5,...,1294.818751,0.368953,0,INDIVIDUAL,18,0,430,126029,22579,0


In [4]:
# Determine the number of unique values in each column.
application_df.dtypes

Loan Amount                       int64
Funded Amount                     int64
Funded Amount Investor          float64
Term                              int64
Batch Enrolled                   object
Interest Rate                   float64
Grade                            object
Sub Grade                        object
Home Ownership                   object
Employment Duration (years)       int64
Verification Status              object
Payment Plan                     object
Loan Title                       object
Debit to Income                 float64
Delinquency - two years           int64
Inquires - six months             int64
Open Account                      int64
Public Record                     int64
Revolving Balance                 int64
Revolving Utilities             float64
Total Accounts                    int64
Initial List Status              object
Total Received Interest         float64
Total Received Late Fee         float64
Recoveries                      float64


In [5]:
# Look at Loan Title value counts for binning
app_type_count = application_df.value_counts('Loan Title')
app_type_count

Loan Title
Credit card refinancing    32061
Debt consolidation         25937
Debt Consolidation          3671
Other                       2565
Home improvement            2308
                           ...  
CC                             6
Personal loan                  5
Getting Ahead                  5
bills                          4
Credit                         4
Name: count, Length: 109, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_count[app_type_count<10].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['Loan Title'] = application_df['Loan Title'].replace(app,"Other")

# Check to make sure binning was successful
application_df['Loan Title'].value_counts()

Loan Title
Credit card refinancing    32061
Debt consolidation         25937
Debt Consolidation          3671
Other                       2674
Home improvement            2308
                           ...  
CONSOLIDATE                   10
vacation                      10
conso                         10
Credit Loan                   10
cards                         10
Name: count, Length: 93, dtype: int64

In [7]:
class_count = application_df.value_counts('Sub Grade')
class_count

Sub Grade
B4    4660
C1    4374
B3    4178
A5    3694
B2    3689
B5    3562
D1    3440
C4    3368
C2    3339
C3    3248
B1    3046
C5    2572
A4    2372
D4    2131
D2    2056
D5    2042
A2    1933
D3    1897
E2    1824
A3    1760
A1    1416
E3    1366
E1    1342
E4    1157
F2     990
F1     861
E5     810
F5     608
F3     602
F4     468
G2     466
G1     385
G5     300
G3     251
G4     159
Name: count, dtype: int64

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(application_df,dtype=int)
application_dummies.head()


,Loan Amount,Funded Amount,Funded Amount Investor,Term,Interest Rate,Employment Duration (years),Debit to Income,Delinquency - two years,Inquires - six months,Open Account,...,Loan Title_loan1,Loan Title_pay off bills,Loan Title_payoff,Loan Title_personal,Loan Title_refi,Loan Title_vacation,Initial List Status_f,Initial List Status_w,Application Type_INDIVIDUAL,Application Type_JOINT
0,10000,32236,12329.36286,59,11.135007,20,16.284758,1,0,13,...,0,0,0,0,0,0,0,1,1,0
1,3609,11940,12191.99692,59,12.237563,4,15.412409,0,0,12,...,0,0,0,0,0,0,1,0,1,0
2,28276,9311,21603.22455,59,12.545884,10,28.137619,0,0,14,...,0,0,0,0,0,0,0,1,1,0
3,11170,6954,17877.15585,59,16.731201,12,18.043730,1,0,7,...,0,0,0,0,0,0,0,1,1,0
4,16890,13226,13539.92667,59,15.008300,5,17.209886,1,3,13,...,0,0,0,0,0,0,0,1,1,0


In [9]:
application_dummies.keys()

Index(['Loan Amount', 'Funded Amount', 'Funded Amount Investor', 'Term',
       'Interest Rate', 'Employment Duration (years)', 'Debit to Income',
       'Delinquency - two years', 'Inquires - six months', 'Open Account',
       ...
       'Loan Title_loan1', 'Loan Title_pay off bills', 'Loan Title_payoff',
       'Loan Title_personal', 'Loan Title_refi', 'Loan Title_vacation',
       'Initial List Status_f', 'Initial List Status_w',
       'Application Type_INDIVIDUAL', 'Application Type_JOINT'],
      dtype='object', length=212)

In [10]:
# Split our preprocessed data into our features and target arrays
X = application_dummies.drop('Loan Status', axis=1).values
y = application_dummies['Loan Status'].values


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Get the input shape
X_train_scaled.shape

(52774, 211)

## Compile, Train and Evaluate the Model

In [13]:
from keras.layers import Dense, Dropout
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train_scaled.shape[1]

hidden_node1 = 100
hidden_node2 = 100
#hidden_node3 = 256


nn_model_1 = tf.keras.models.Sequential()

# First hidden layer
nn_model_1.add(tf.keras.layers.Dense(units=hidden_node1, activation='relu', input_dim = input_features))

# Second hidden layer
nn_model_1.add(tf.keras.layers.Dense(units=hidden_node2, activation='relu'))

# Third hidden layer
#nn_model_1.add(tf.keras.layers.Dense(units=hidden_node3, activation='relu'))
#nn_model_1.add(Dropout(0.5))

# Output layer
nn_model_1.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               21200     
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 31,401
Trainable params: 31,401
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn_model_1.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [15]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define checkpoints variables
os.makedirs('/Users/batoogz/Desktop/Module Challenges/project-4-web-scrapers/Resources/Raph_Model_Files/MyModels/checkpoints/', exist_ok=True)
checkpoint_path= '/Users/batoogz/Desktop/Module Challenges/project-4-web-scrapers/Resources/Raph_Model_Files/MyModels/checkpoints/weights.{epoch}.hdf5'

# Create callback
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              verbose=1,
                              save_weights_only=True,
                              save_freq='epoch',
                              period=5)

In [18]:
fit_model_1 = nn_model_1.fit(X_train_scaled, y_train, epochs=60, batch_size=32, validation_split=0.2, callbacks=[cp_callback])

Epoch 1/60
1320/1320 [==============================] - 1s 613us/step - loss: 0.0265 - accuracy: 0.9919 - val_loss: 1.9224 - val_accuracy: 0.8600
Epoch 2/60
1320/1320 [==============================] - 1s 598us/step - loss: 0.0193 - accuracy: 0.9941 - val_loss: 2.0483 - val_accuracy: 0.8751
Epoch 3/60
1320/1320 [==============================] - 1s 589us/step - loss: 0.0233 - accuracy: 0.9924 - val_loss: 1.9514 - val_accuracy: 0.8582
Epoch 4/60
1320/1320 [==============================] - 1s 581us/step - loss: 0.0206 - accuracy: 0.9932 - val_loss: 1.9160 - val_accuracy: 0.8467
Epoch 5/60
1222/1320 [==========================>...] - ETA: 0s - loss: 0.0166 - accuracy: 0.9940
Epoch 5: saving model to /Users/batoogz/Desktop/Module Challenges/project-4-web-scrapers/Resources/Raph_Model_Files/MyModels/checkpoints/weights.5.hdf5
1320/1320 [==============================] - 1s 579us/step - loss: 0.0176 - accuracy: 0.9938 - val_loss: 1.9801 - val_accuracy: 0.8576
Epoch 6/60
1320/1320 [=========

In [17]:
df_training_record = pd.DataFrame(fit_model_1.history)
df_training_record


,loss,accuracy,val_loss,val_accuracy
0,0.319333,0.909472,0.305418,0.911037
1,0.300617,0.910301,0.309115,0.910280
2,0.293347,0.910538,0.310143,0.911227
3,0.286379,0.910277,0.316846,0.911037
4,0.275529,0.910585,0.326126,0.910469
5,0.262880,0.911533,0.337751,0.907342
6,0.247736,0.913309,0.347865,0.908764
7,0.232759,0.916767,0.364310,0.905732
8,0.216979,0.920154,0.381622,0.898437
9,0.201443,0.926479,0.405197,0.893415


In [19]:
#Save Training History for documentation
from pathlib import Path
filepath = Path('/Users/batoogz/Desktop/Module Challenges/project-4-web-scrapers/Resources/Raph_Model_Files/Saved Records/df_training_record.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df_training_record.to_csv(filepath)

In [20]:
# Save H5 model files to drive
nn_model_1.save('/Users/batoogz/Desktop/Module Challenges/project-4-web-scrapers/Resources/Raph_Model_Files/Saved_Models/raph-model.h5')
print("Successfully saved H5File model to path")


Successfully saved H5File model to path


In [58]:
# install tfjs if not yet installed
#!pip install tensorflowjs


In [24]:
#!mkdir MyModels
!tensorflowjs_converter --input_format keras \/content/drive/MyDrive/Colab_Notebooks/Saved_Models/raph-model.h5 \content/drive/MyDrive/Colab_Notebooks/MyModels/

2023-10-10 23:02:08.857934: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-10 23:02:08.858454: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-10 23:02:08.858562: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-10 23:02:12.131507: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [21]:
#Looking up the shape of X_test
X_test.shape

(17592, 211)

In [22]:
#randomize number base on the length of X_test
from random import randint

randomizer = randint(1, len(X_test))
print(randomizer)

4212


In [23]:
#make 1 prediction based on randomized X_test Row
prediction =  nn_model_1.predict(X_test[[randomizer]])
print("X_test row %s=%s, Predicted=%s" % (randomizer,X_test[[randomizer]], prediction[0]))

1/1 [==============================] - 0s 45ms/step
X_test row 4212=[[2.30800000e+03 2.31400000e+04 2.09055379e+04 5.90000000e+01
  1.05866792e+01 1.00000000e+01 3.13570422e+01 0.00000000e+00
  0.00000000e+00 8.00000000e+00 0.00000000e+00 1.09000000e+03
  3.39352704e+01 1.70000000e+01 3.03884664e+03 4.06757960e-02
  2.65422713e+00 3.83561006e-01 0.00000000e+00 5.60000000e+01
  0.00000000e+00 5.00000000e+01 2.93841000e+05 7.68700000e+03
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
  

In [24]:
#Extract X_test, y_test, X_train, and y_train

x_test_df = pd.DataFrame(X_test)
y_test_df = pd.DataFrame(y_test)
X_train_df = pd.DataFrame(X_train)
y_train_df = pd.DataFrame(y_train)




In [65]:
x_test_df

,0,1,2,3,4,5,6,7,8,9,...,201,202,203,204,205,206,207,208,209,210
0,16339.0,12883.0,9163.479779,58.0,11.182507,6.0,32.130816,0.0,0.0,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,10381.0,10362.0,15354.354110,58.0,10.204524,7.0,17.181214,1.0,0.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,9866.0,11243.0,8071.290521,58.0,9.811078,9.0,15.013815,0.0,0.0,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,5511.0,33029.0,9573.203584,59.0,12.304106,6.0,22.194441,1.0,0.0,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,14462.0,15704.0,13229.566710,59.0,10.988328,9.0,19.750259,1.0,0.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17587,12389.0,3098.0,9297.266815,59.0,11.902899,4.0,10.799116,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
17588,20036.0,11447.0,19119.243260,59.0,15.731963,4.0,8.686723,0.0,0.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
17589,17698.0,8909.0,24661.487100,58.0,12.853400,5.0,23.243128,0.0,0.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
17590,5695.0,34804.0,13351.961040,59.0,12.072148,4.0,33.378423,0.0,0.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [66]:
#Save all DF to CSV

from pathlib import Path
#for X_test
filepath = Path('/Users/batoogz/Desktop/Module Challenges/project-4-web-scrapers/Resources/Raph_Model_Files/Splits/X_test2.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
x_test_df.to_csv(filepath)

#for_y_test
filepath = Path('/Users/batoogz/Desktop/Module Challenges/project-4-web-scrapers/Resources/Raph_Model_Files/Splits/y_test2.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
y_test_df.to_csv(filepath)

#for x_train
filepath = Path('/Users/batoogz/Desktop/Module Challenges/project-4-web-scrapers/Resources/Raph_Model_Files/Splits/X_train2.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
X_train_df.to_csv(filepath)

#for y_train
filepath = Path('/Users/batoogz/Desktop/Module Challenges/project-4-web-scrapers/Resources/Raph_Model_Files/Splits/y_train2.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
y_train_df.to_csv(filepath)